In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm, Normalize
import seaborn as sb
import os
import sys
sys.setrecursionlimit(1000000)
from scipy.stats import spearmanr
import pathlib


Load barcodes and LCM registration files and assign counts to areas

In [ ]:
directory = pathlib.PurePath('/camp/lab/znamenskiyp/home/shared/projects/turnerb_MAPseq/A1_MAPseq/FIAA32.6a/Sequencing/Processed_data/BC_split/temp/increased_cutoff/')
barcodes_across_sample = pd.read_pickle(directory/'raw_barcodes_across_sample_higher_cutoff.pkl')
#load registration files containing volume of each brain area within each sample and which RT primer corresponds to which sample name
lcm_reg_dir = pathlib.PurePath('/nemo/lab/znamenskiyp/home/shared/code/MAPseq_processing/AC_MAPseq/Brain1_FIAA32.6a/LCM_registration')
#_3dareas = '/camp/lab/znamenskiyp/home/shared/code/MAPseq_processing/AC_MAPseq/Brain1_FIAA32.6a/LCM_registration/3D_areas_in_sample.csv'
areas = pd.read_csv(lcm_reg_dir/'3d_areas.csv')
RTtosample = pd.read_csv(lcm_reg_dir/'RTprimer_tosample.csv')
areas = areas.merge(RTtosample, how='inner', on='sample')
areas.sort_values("RT_primer", inplace=True)

In [ ]:
#plot heatmap of barcodes against samples
fig, ax = plt.subplots(figsize=(60, 10))
sb.heatmap(barcodes_across_sample, norm=LogNorm())
plt.show()

In [ ]:
barcodes_across_sample

In [ ]:
plt.hist(np.sum(barcodes_across_sample.to_numpy()>5, axis=1), bins=np.arange(5))

In [ ]:
plt.hist(barcodes_across_sample.to_numpy().flatten(), bins=100)
plt.yscale('log')
plt.xlabel('Frequency')
plt.ylabel('Barcode UMI count per sample')
plt.title('Distribution of UMIs per barcode per sample')

In [ ]:
#look at BC distribution in negative control
neg =barcodes_across_sample[[1, 2, 3, 4, 5]].to_numpy().flatten()
plt.hist(neg.flatten(), bins=100)
#plt.yscale('log')
plt.xlabel('Frequency')
plt.ylabel('Barcode UMI count per sample')
plt.title('Distribution of UMIs per barcode in negative control')


In [ ]:
neg1 = pd.DataFrame(neg).dropna()
neg1.value_counts()

In [ ]:
#look at barcode distribution in sample sites vs neg control
neg_sites =[1, 2, 3, 4, 5]
fig, ax1 = plt.subplots()
sample_sites = barcodes_across_sample.drop(columns=neg_sites) 
plt.hist(sample_sites.to_numpy().flatten(), bins=100, label ='sample sites')
plt.hist(neg.flatten(), bins=np.arange(5000), label = 'negative control')
plt.yscale('log')
plt.xlabel('Frequency')
plt.ylabel('Barcode UMI count per sample')
plt.title('Distribution of UMIs per barcode per sample')

plt.legend(loc='upper right')
plt.show()

In [ ]:
#plots the histogram
fig, ax1 = plt.subplots()
ax1.hist([sample_sites.to_numpy().flatten(),neg.flatten()], histtype='step', linewidth=2, bins=100, label=['sample sites', 'negative control'])
#ax1.set_xlim(-10,10)
ax1.set_ylabel("Frequency")
ax1.set_xlabel("UMIs per barcode")
plt.tight_layout()
plt.yscale('log')
plt.legend(loc='upper right')
plt.title('Distribution of UMIs per barcode per sample')
plt.show()

In [ ]:
#set max barcode to one
newbcmatrix = pd.DataFrame(columns = barcodes_across_sample.columns)
for i, row in barcodes_across_sample.iterrows():
    newrow= pd.DataFrame(barcodes_across_sample.loc[i]/barcodes_across_sample.loc[i].max())
    newbcmatrix = pd.concat([newbcmatrix, newrow.T])


In [ ]:
fig, ax = plt.subplots(figsize=(60, 10))
sb.heatmap(newbcmatrix, cmap='Blues', norm=LogNorm())
plt.show()

In [ ]:
#spike-in normalisation, generate table of spike counts per sample
spike_counts = pd.DataFrame(columns=["sample", "spike_count"])
for sample in os.listdir(directory):
    if sample.startswith("spikecounts"):
        sample_name = sample.split("spikecounts_", 1)
        sample_name = sample_name[1][: -len(".csv")]
        sample_num = float(sample_name[2 :])
        sample_reading = pd.read_csv(directory/sample)
        sample_reading["counts"] = sample_reading["counts"].astype("int")
        sum_counts = sample_reading["counts"].sum()
        new_row = pd.DataFrame(
            {"sample": sample_num, "spike_count": sum_counts}, index=[0]
        )
        spike_counts = pd.concat([spike_counts, new_row])


In [ ]:
#only look at most abundant barcodes by taking min values for source sites
neg =barcodes_across_sample[[40, 41, 42, 43, 49, 50, 51, 52]].to_numpy().flatten()
plt.hist(neg.flatten(), bins=50)
plt.yscale('log')
plt.xlabel('Barcode UMI count per sample')
plt.ylabel('Frequency')
plt.title('Distribution of UMIs per barcode in source sites')

In [ ]:
#select rows based on min count at source sites
source_min = 10
barcodes_norm_sub1 = barcodes_across_sample.loc[(barcodes_across_sample[40] >= source_min)]
barcodes_norm_sub2 = barcodes_across_sample.loc[(barcodes_across_sample[41] >= source_min)]
barcodes_norm_sub3 = barcodes_across_sample.loc[(barcodes_across_sample[42] >= source_min)]
barcodes_norm_sub4 = barcodes_across_sample.loc[(barcodes_across_sample[43] >= source_min)]
barcodes_norm_sub5 = barcodes_across_sample.loc[(barcodes_across_sample[49] >= source_min)]
barcodes_norm_sub6 = barcodes_across_sample.loc[(barcodes_across_sample[50] >= source_min)]
barcodes_norm_sub7 = barcodes_across_sample.loc[(barcodes_across_sample[51] >= source_min)]
barcodes_norm_sub8 = barcodes_across_sample.loc[(barcodes_across_sample[52] >= source_min)]
newdf =pd.concat([barcodes_norm_sub1, barcodes_norm_sub2])
newdf =pd.concat([newdf, barcodes_norm_sub3])
newdf =pd.concat([newdf, barcodes_norm_sub4])
newdf =pd.concat([newdf, barcodes_norm_sub5])
newdf =pd.concat([newdf, barcodes_norm_sub6])
newdf =pd.concat([newdf, barcodes_norm_sub7])
newdf =pd.concat([newdf, barcodes_norm_sub8])
newdf = newdf[~newdf.index.duplicated(keep='first')] #remove duplicate barcodes


In [ ]:
newdf

In [ ]:
barcodes_across_sample = newdf

In [ ]:
#drop samples that contain spike count less than 10, as RT likely failed for these samples
min_spike = 1500
spike_thresholded = spike_counts[spike_counts['spike_count'] >= min_spike]
areas_dropped= areas[areas['RT_primer'].isin(spike_thresholded['sample']) == False].RT_primer
areas= areas[areas['RT_primer'].isin(spike_thresholded['sample']) == True]
barcodes_across_sample = barcodes_across_sample.drop(columns=np.array(areas_dropped))    

In [ ]:
#also drop sample 5 that doesn't have reg info
areas = areas.drop([4])
barcodes_across_sample =barcodes_across_sample.drop(columns=[5])

In [ ]:
plt.hist(spike_counts["spike_count"], bins=30)
plt.title('Spike-in Count Distribution', fontsize=12)
plt.axvline(x = 1500, color = "Black", label = "cut-off")
plt.legend(loc = 'upper right')
plt.xlabel('Counts')
plt.ylabel('# Samples')

In [ ]:
areas_dropped

In [ ]:
#now remove any barcodes with a count of 1, then remove barcodes that don't have a count anywhere.\
barcodes_across_sample = barcodes_across_sample.replace(1,0)
barcodes_across_sample.fillna(0,inplace=True)
barcodes_across_sample = barcodes_across_sample.loc[~(barcodes_across_sample==0).all(axis=1)]


In [ ]:
bla = barcodes_across_sample[1][barcodes_across_sample[1]>0]

In [ ]:
barcodes_across_sample

In [ ]:
areas[areas['RT_primer']==56]

In [ ]:
#plot heatmap of barcodes against samples
fig, ax = plt.subplots(figsize=(60, 10))
sb.heatmap(barcodes_across_sample, norm=LogNorm())
plt.show()

In [ ]:
spike_thresholded

In [ ]:
#normalise counts by spike-in counts
lowest = min(spike_thresholded["spike_count"])
spike_thresholded["normalisation_factor"] = spike_thresholded["spike_count"] / lowest
#spike_thresholded= spike_thresholded.sort_values("sample", inplace=True)
spike_thresholded =spike_thresholded.set_index('sample')
spike_thresholded.sort_index(inplace=True)
norm = spike_thresholded['normalisation_factor'].T
barcodes_across_sample = barcodes_across_sample.div(norm, axis='columns')

barcodes_across_sample.fillna(0,inplace=True)
#plt heatmap of barcode matrix after spike normalisation
fig, ax = plt.subplots(figsize=(60, 10))
sb.heatmap(barcodes_across_sample, cmap='Blues', norm=LogNorm())
plt.show()

In [ ]:
barcodes_across_sample

In [ ]:
areas.loc[28].sum()

In [ ]:
#load ROI from registration data, and plot striatum reads against coordinates
reg_dir = pathlib.Path('/nemo/lab/znamenskiyp/home/shared/projects/turnerb_MAPseq/A1_MAPseq/FIAA32.6a/LCM_registration/allenccf/allen_ccf_coord')
ROI = pathlib.Path('/nemo/lab/znamenskiyp/home/shared/projects/turnerb_MAPseq/A1_MAPseq/FIAA32.6a/LCM_registration/rois')

In [ ]:
CAU_samples = [8, 9, 13, 19, 29]


In [ ]:
barcodes_across_sample[CAU_samples]
#plt heatmap of barcode matrix after spike normalisation
fig, ax = plt.subplots(figsize=(60, 10))
sb.heatmap(barcodes_across_sample[CAU_samples], cmap='Blues', norm=LogNorm())
plt.show()

In [ ]:
#set max barcode to one
CAU =barcodes_across_sample[CAU_samples]
newbcmatrix = pd.DataFrame(columns = CAU.columns)
for i, row in CAU.iterrows():
    newrow= pd.DataFrame(CAU.loc[i]/CAU.loc[i].max())
    newbcmatrix = pd.concat([newbcmatrix, newrow.T])
fig, ax = plt.subplots(figsize=(60, 10))
sb.heatmap(newbcmatrix, cmap='Blues')
plt.show()

In [ ]:
pd.set_option('display.max_rows', 90)
pd.set_option('display.max_columns', 160)

In [ ]:
#source_sites = [40, 42, 43, 49, 50, 51, 52] #removed 41, as this is already removed
#barcodes_across_sample.drop(source_sites, axis=1, inplace=True)
#areas.drop(index=[ source - 1 for source in source_sites ], inplace=True)

group_areas = {
    'SC': ['SCdg', 'SCdw', 'SCig', 'SCiw', 'SCop', 'SCsg', 'SCzo'],
    
    'IC': ['ICc', 'ICd', 'ICe'],
    'SSp': ['SSp-bfd', 'SSp-ll', 'SSp-m', 'SSp-n', 'SSp-tr', 'SSp-ul', 'SSp-un'],
    'contra': areas.filter(like="Contra").columns,
    'striatum': ['CP', 'STR', 'ACB'],
    'pons': ['SOCm', 'SOCl', 'POR', 'PRNr', 'PRNc', 'TRN', 'P', 'P-mot']
}
for group, columns in group_areas.items():
    areas[group] = areas.filter(items=columns).sum(axis=1)
    areas = areas.drop(columns, axis=1)
    
areas_only = areas.drop(['sample', 'RT_primer', 'ar', 'bic', 'bsc', 'ccb', 'ccb', 'ccg', 'cing', 'cpd', 'csc', 'cst', 'ec', 'fa', 'fi',
    'fiber tracts', 'fp', 'll', 'mcp', 'ml', 'onl', 'or', 'py', 'root', 'sctv', 'scwm', 'tb', 'CTXsp', 'act', 'alv', 'amc', 'cic', 'TH'], axis=1)


In [ ]:
areas_only

In [ ]:
areas

In [ ]:
#source_sites = [40, 42, 43, 49, 50, 51, 52] #removed 41, as this is already removed
#barcodes_across_sample.drop(source_sites, axis=1, inplace=True)
#areas.drop(index=[ source - 1 for source in source_sites ], inplace=True)

group_areas = {
    'SC': ['SCdg', 'SCdw', 'SCig', 'SCiw', 'SCop', 'SCsg', 'SCzo'],
    
    'IC': ['ICc', 'ICd', 'ICe'],
    'SSp': ['SSp-bfd', 'SSp-ll', 'SSp-m', 'SSp-n', 'SSp-tr', 'SSp-ul', 'SSp-un'],
    'contra': areas.filter(like="Contra").columns,
    'striatum': ['CP', 'STR', 'ACB'],
    'pons': ['SOCm', 'SOCl', 'POR', 'PRNr', 'PRNc', 'TRN', 'P', 'P-mot']
}

for group, columns in group_areas.items():
    areas[group] = areas.filter(items=columns).sum(axis=1)
    areas = areas.drop(columns, axis=1)
    
areas_only = areas.drop(['sample', 'RT_primer', 'ar', 'bic', 'bsc', 'ccb', 'ccb', 'ccg', 'cing', 'cpd', 'csc', 'cst', 'ec', 'fa', 'fi',
    'fiber tracts', 'fp', 'll', 'mcp', 'ml', 'onl', 'or', 'py', 'root', 'sctv', 'scwm', 'tb', 'CTXsp', 'act', 'alv', 'amc', 'cic', 'TH'], axis=1)

areas_only = areas_only.loc[:, np.sum(areas_only, axis=0)>0]
areas_matrix = areas_only.to_numpy()
areas_matrix /= np.sum(areas_matrix, axis=0)
barcodes_across_sample.fillna(0,inplace=True)
barcodes_matrix = barcodes_across_sample.to_numpy()
barcodes_matrix[np.isnan(barcodes_matrix)] = 0
total_projection_strength = np.sum(barcodes_matrix, axis=1)
barcodes_matrix /= total_projection_strength[:, np.newaxis]

barcodes_matrix = barcodes_matrix[total_projection_strength>0, :]

In [ ]:
from sklearn.linear_model import LinearRegression, Lasso

mdl = LinearRegression(fit_intercept=False, positive=True)
mdl.fit(areas_matrix, barcodes_matrix.T)

In [ ]:
#raw, not spike normalised, with higher cutoff
plt.figure(figsize=(20,70))
df = pd.DataFrame(mdl.coef_[:15000,:], columns=areas_only.columns)
sb.clustermap(df.T, vmax=0.3, dendrogram_ratio=[0.1, 0.1], yticklabels=True)


In [ ]:
#pre-cutoff with spike normalised
plt.figure(figsize=(20,70))
df = pd.DataFrame(mdl.coef_[:15000,:], columns=areas_only.columns)
sb.clustermap(df.T, vmax=0.2, dendrogram_ratio=[0.1, 0.1], yticklabels=True)


In [ ]:
plt.figure(figsize=(20, 5))
sb.barplot(df)
plt.xticks(rotation=90)

In [ ]:
#remove barcodes that are only seen in one sample (NB this is not needed, I initially put in as a QC)
barcodes_across_sample['samplesnotin'] =0
for index, row in barcodes_across_sample.iterrows():
    barcodes_across_sample['samplesnotin'].iloc[index]=(row.isna().sum())
barcodes_across_sample = barcodes_across_sample[barcodes_across_sample['samplesnotin']<90]
barcodes_across_sample = barcodes_across_sample.drop('samplesnotin', axis=1)


In [ ]:
#remove NaN
barcodes_across_sample = barcodes_across_sample.fillna(0)
#set min val to 1
barcodes_across_sample= barcodes_across_sample.reset_index(drop=True)
for index, row in barcodes_across_sample.iterrows():
    bla = np.array(row)
    smallest = np.min(bla[np.nonzero(bla)])
    barcodes_across_sample.iloc[[index]]=row/smallest

In [ ]:
#select rows based on min count at source sites
barcodes_norm_sub1 = barcodes_across_sample.loc[(barcodes_across_sample[40] >= 20)]
barcodes_norm_sub2 = barcodes_across_sample.loc[(barcodes_across_sample[41] >= 20)]
barcodes_norm_sub3 = barcodes_across_sample.loc[(barcodes_across_sample[42] >= 20)]
barcodes_norm_sub4 = barcodes_across_sample.loc[(barcodes_across_sample[43] >= 20)]
barcodes_norm_sub5 = barcodes_across_sample.loc[(barcodes_across_sample[49] >= 20)]
barcodes_norm_sub6 = barcodes_across_sample.loc[(barcodes_across_sample[50] >= 20)]
barcodes_norm_sub7 = barcodes_across_sample.loc[(barcodes_across_sample[51] >= 20)]
barcodes_norm_sub8 = barcodes_across_sample.loc[(barcodes_across_sample[52] >= 20)]
newdf =pd.concat([barcodes_norm_sub1, barcodes_norm_sub2])
newdf =pd.concat([newdf, barcodes_norm_sub3])
newdf =pd.concat([newdf, barcodes_norm_sub4])
newdf =pd.concat([newdf, barcodes_norm_sub5])
newdf =pd.concat([newdf, barcodes_norm_sub6])
newdf =pd.concat([newdf, barcodes_norm_sub7])
newdf =pd.concat([newdf, barcodes_norm_sub8])
newdf = newdf[~newdf.index.duplicated(keep='first')] #remove duplicate barcodes

In [ ]:
#plot heatmap showing barcodes in source site
fig, ax = plt.subplots(figsize=(60, 10))
sb.heatmap(barcodes_across_sample, norm=LogNorm())
plt.show()

In [ ]:
#plot heatmap showing barcodes in source site with minimum thresholds
fig, ax = plt.subplots(figsize=(60, 10))
sb.heatmap(newdf, norm=LogNorm())
plt.show()

In [ ]:
#now remove columns containing source sites and negative control
newdf = newdf.drop([40, 41, 42, 43, 49, 50, 51, 52, 1, 2, 3, 4, 5], axis=1)


In [ ]:
total['sum'] = areas_only.sum(axis=1)
total

In [ ]:
#create a dataframe of the fractions of each brain area contained within each sample
total = pd.DataFrame()
total['sum'] = areas_only.sum(axis=1)
areasFrac = pd.DataFrame(columns=areas_only.columns)
for i, row in areas_only.iterrows():
    newrow = row/total['sum'].iloc[i]
    areasFrac =areasFrac.append(newrow)

In [ ]:
areas= areas.drop(['ar', 'bic', 'bsc', 'ccb', 'ccb', 'ccg', 'cing', 'cpd', 'csc', 'cst', 'ec', 'fa', 'fi',
    'fiber tracts', 'fp', 'll', 'mcp', 'ml', 'onl', 'or', 'py', 'root', 'sctv', 'scwm', 'tb', 'CTXsp', 'act', 'alv', 'amc', 'cic', 'TH'], axis=1)

In [ ]:
areas = areas.drop(areas.iloc[4])

In [ ]:
areasFrac

In [ ]:
#for each barcode, create a matrix of BC count for regions in a sample based on amount of each region in LCM (makes assumption of equal BC distribution)
bc_matrix = np.zeros(shape=((len(newdf), (len(areas_only.columns)))))
bc_matrix = pd.DataFrame(data= bc_matrix, columns=areas_only.columns, index=newdf.index)
for i, row in newdf.iterrows():
    bc_matrix1 =pd.DataFrame(columns=areas_only.columns)
    for samplename in newdf.columns:
        ind = areas.index[areas['RT_primer']==samplename].tolist()
        fractionC = areasFrac.iloc[ind[0]]*row.loc[samplename]
        bc_matrix1 = bc_matrix1.append(fractionC)
    for region in bc_matrix1.columns:
        bc_matrix.at[i, region] = bc_matrix1[region].sum()/areas_only[region].sum()
#bc_matrix.to_pickle(lcm_reg_dir/'bc_matrix_lcm_2.pkl')

In [ ]:
#load bc_matrix (containing counts in each region for each barcode) if don't want to repeat above  
bc_matrix = pd.read_pickle(lcm_reg_dir/'bc_matrix_lcm_2.pkl')



In [ ]:
#remove columns that are all zeros, and rows that are all zeros
for column in bc_matrix.columns:
    if bc_matrix[column].sum() == 0:
        bc_matrix.drop([column], axis=1, inplace=True)
bc_matrix = bc_matrix.loc[~(bc_matrix==0).all(axis=1)]

In [ ]:
# perform hierarchial clustering of all barcodes across samples
sb.clustermap(bc_matrix, metric='euclidean', standard_scale=0, cmap="Blues", figsize=(60, 10))

Potentially may want to threshold to minimum barcode counts? I haven't but might be useful set a minimum

In [ ]:
#threshold minimum value of counts/cm3 to zero
threshold = 0.0000001
bc_matrix_thresholded = pd.DataFrame(np.where(bc_matrix > threshold, 0, bc_matrix))
#remove columns that are all zeros, and rows that are all zeros
for column in bc_matrix_thresholded.columns:
    if bc_matrix_thresholded[column].sum() == 0:
        bc_matrix_thresholded.drop([column], axis=1, inplace=True)
bc_matrix_thresholded = bc_matrix_thresholded.loc[~(bc_matrix==0).all(axis=1)]
#plot heatmap showing barcodes in source site with minimum thresholds
fig, ax = plt.subplots(figsize=(60, 10))
sb.heatmap(bc_matrix_thresholded, norm=LogNorm())
plt.show()

Looking at barcode distribution across visual areas only

In [ ]:
#before selecting subset of areas, set max row projection strength to 1, so preserve relative projection strengths of bc
newbcmatrix = pd.DataFrame(columns = bc_matrix.columns)
for i, row in bc_matrix.iterrows():
    newrow= pd.DataFrame(bc_matrix.loc[i]/bc_matrix.loc[i].max())
    newbcmatrix = pd.concat([newbcmatrix, newrow.T])

In [ ]:
#now take only the regions that contain visual areas
visareas= [col for col in newbcmatrix if col.startswith('VIS') and col.startswith('VISC') == False] +  [col for col in newbcmatrix if col.startswith('Contra-VIS')and col.startswith('Contra-VISC') == False]
reg = newbcmatrix.loc[:,visareas]




In [ ]:
#remove rows and columns containing only zeros
for column in reg.columns:
    if reg[column].sum() == 0:
        reg.drop([column], axis=1, inplace=True)
reg = reg.loc[~(reg==0).all(axis=1)]

In [ ]:
#perform hierarchial clustering of visual areas only
sb.clustermap(reg, metric='euclidean', cmap="Blues", figsize=(30, 10))

(can ignore) looking at qPCR as potential QC check

In [ ]:
areas = pd.read_csv(lcm_reg_dir/'3d_areas.csv')
RTtosample = pd.read_csv(lcm_reg_dir/'RTprimer_tosample.csv')
areas = areas.merge(RTtosample, how='inner', on='sample')
areas.sort_values("RT_primer", inplace=True)
areas.drop('RT_primer', axis=1)

In [ ]:
#plot of qPCR beta actin values against volume for potentially using as QC against sample quality
qPCR = pd.read_csv('/camp/lab/znamenskiyp/home/shared/projects/turnerb_MAPseq/A1_MAPseq/FIAA32.6a/qPCR/qPCR_FIAA326a.csv') 
qPCR['B-act_amount'] = np.power(1.585,(-(qPCR['B-actin Ct'])))
qPCR['vol'] = 0
qPCR['vol'] = areas.sum(axis=1)    
#for i, row in qPCR.iterrows():
 #   ind = areas.index[areas['RT_primer']==qPCR.loc[i, 'RT primer']].tolist()
  #  qPCR.at[i, 'vol'] = total.iloc[ind[0]]
#qPCR = qPCR.drop([4]) #remove row with no volume
qPCR['logVol'] = np.log(qPCR['vol'])
qPCR['logBetaAct'] = np.log(qPCR['B-act_amount'])
sb.lmplot(data= qPCR, x='logVol', y='logBetaAct')
corr, _ = spearmanr(qPCR["vol"], qPCR["B-act_amount"])
print('Spearmans correlation: %.3f' % corr)

In [ ]:
import scipy as sp
import scipy.stats as stats
import matplotlib.pyplot as plt
def plot_ci_manual(t, s_err, n, x, x2, y2, ax=None):
    """Return an axes of confidence bands using a simple approach.
    
    Notes
    -----
    .. math:: \left| \: \hat{\mu}_{y|x0} - \mu_{y|x0} \: \right| \; \leq \; T_{n-2}^{.975} \; \hat{\sigma} \; \sqrt{\frac{1}{n}+\frac{(x_0-\bar{x})^2}{\sum_{i=1}^n{(x_i-\bar{x})^2}}}
    .. math:: \hat{\sigma} = \sqrt{\sum_{i=1}^n{\frac{(y_i-\hat{y})^2}{n-2}}}
    
    References
    ----------
    .. [1] M. Duarte.  "Curve fitting," Jupyter Notebook.
       http://nbviewer.ipython.org/github/demotu/BMC/blob/master/notebooks/CurveFitting.ipynb
    
    """
    if ax is None:
        ax = plt.gca()
    
    ci = t * s_err * np.sqrt(1/n + (x2 - np.mean(x))**2 / np.sum((x - np.mean(x))**2))
    ax.fill_between(x2, y2 + ci, y2 - ci, color="#b9cfe7", edgecolor="none")

    return ax


def plot_ci_bootstrap(xs, ys, resid, nboot=500, ax=None):
    """Return an axes of confidence bands using a bootstrap approach.

    Notes
    -----
    The bootstrap approach iteratively resampling residuals.
    It plots `nboot` number of straight lines and outlines the shape of a band.
    The density of overlapping lines indicates improved confidence.

    Returns
    -------
    ax : axes
        - Cluster of lines
        - Upper and Lower bounds (high and low) (optional)  Note: sensitive to outliers

    References
    ----------
    .. [1] J. Stults. "Visualizing Confidence Intervals", Various Consequences.
       http://www.variousconsequences.com/2010/02/visualizing-confidence-intervals.html

    """ 
    if ax is None:
        ax = plt.gca()

    bootindex = sp.random.randint

    for _ in range(nboot):
        resamp_resid = resid[bootindex(0, len(resid) - 1, len(resid))]
        # Make coeffs of for polys
        pc = sp.polyfit(xs, ys + resamp_resid, 1)                   
        # Plot bootstrap cluster
        ax.plot(xs, sp.polyval(pc, xs), "b-", linewidth=2, alpha=3.0 / float(nboot))

    return ax

In [ ]:
def equation(a, b):
    """Return a 1D polynomial."""
    return np.polyval(a, b) 


x = qPCR['logVol']
y = qPCR['logBetaAct']
p, cov = np.polyfit(x, y, 1, cov=True)                     # parameters and covariance from of the fit of 1-D polynom.
y_model = equation(p, x)                                   # model using the fit parameters; NOTE: parameters here are coefficients

# Statistics
n = qPCR['logBetaAct'].size                                           # number of observations
m = p.size                                                 # number of parameters
dof = n - m                                                # degrees of freedom
t = stats.t.ppf(0.975, n - m)                              # t-statistic; used for CI and PI bands

# Estimates of Error in Data/Model
resid = y - y_model                                        # residuals; diff. actual data from predicted values
chi2 = np.sum((resid / y_model)**2)                        # chi-squared; estimates error in data
chi2_red = chi2 / dof                                      # reduced chi-squared; measures goodness of fit
s_err = np.sqrt(np.sum(resid**2) / dof)                    # standard deviation of the error

# Plotting --------------------------------------------------------------------
fig, ax = plt.subplots(figsize=(8, 6))

# Data
ax.plot(
    x, y, "o", color="#b9cfe7", markersize=5, 
    markeredgewidth=0.1, markeredgecolor="#0047AB", markerfacecolor="#0047AB"
)

# Fit
ax.plot(x, y_model, "-", color="0.1", linewidth=1.5, alpha=0.5, label="Fit")  

x2 = np.linspace(np.min(x), np.max(x), 100)
y2 = equation(p, x2)

# Confidence Interval (select one)
plot_ci_manual(t, s_err, n, x, x2, y2, ax=ax)
#plot_ci_bootstrap(x, y, resid, ax=ax)
   
# Prediction Interval
pi = t * s_err * np.sqrt(1 + 1/n + (x2 - np.mean(x))**2 / np.sum((x - np.mean(x))**2))   
ax.fill_between(x2, y2 + pi, y2 - pi, color="None", linestyle="--")
ax.plot(x2, y2 - pi, "--", color="0.5", label="95% Prediction Limits")
ax.plot(x2, y2 + pi, "--", color="0.5")
plt.xlabel('log vol')
plt.ylabel('log b-actin')
#plt.show()

In [ ]:
#remove samples that are outside 95% prediction limit of b-act/vol regression fit
low_bact_corr = [67, 39, 25, 5, 44] #43 already removed in spike-in
barcodes_across_sample.drop(low_bact_corr, axis=1, inplace=True)
areas = areas[~areas['RT_primer'].isin(low_bact_corr)]